Decorators:

    In Python, functions are the first class objects.
        which means that

        Functions
            are objects:
            they can be referenced to,
            passed to a variable and 
            returned from other functions as well.
        
        Functions can be defined inside another function and can
        also be passed as argument to another function. 

        Decorators allow programmers to modify the behavior fo function or class.

        Decorators allow us to wrap another functionin order to extend the behavior of wrapped function, without permanently modifying it.

    Any generic functionality you can 'tack on' to an existing class or function's behavior makes a great use case for decoration.

    This includes:

        logging
        enforcing access control and authentication,
        instrumentation and timing functions
        rate-limiting
        caching and more
    First you have to know or remember that function names are references to functions and that we can assign multiple names to the same function:

    

In [23]:
from IPython.display import Image

In [24]:
def success(x):
    return x + 1

In [25]:
success(10)

11

In [26]:
successor = success

In [27]:
successor(20)

21

This means that we have 2 names i.e "sucess" and "successor" for the same function.

The next important fact is that we can delete either "success" or "successor" without deleting the function itself.

In [28]:
del success 
successor(10)

11

Functions inside Functions

In [30]:
def f():
    
    def g():
        print("Hi, It's me 'g'")
        print("Thanks for calling me")
        
    print("This is function 'f'")
    print("I am calling 'g' now:\n")
    
    g()

In [31]:
f()

This is function 'f'
I am calling 'g' now:

Hi, It's me 'g'
Thanks for calling me


Another example using "proper" return statements in the functions:



In [32]:
def temperature(t):
    def celsius2fahrenheit(x):
        return 9 * x / 5 + 32
    
    result = "It's " + str(celsius2fahrenheit(t)) + " degrees!"
    return result

In [33]:
print(temperature(20))

It's 68.0 degrees!


Example - factorial

In [35]:
def factorial(n):
    """Calculates the factoral of n,
        n should be an integer and n <= 0"""
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)    

In [36]:
factorial(5)

120

What happens if someone passes a negative value or a float number to this function?

In [37]:
def factorial(n):
    """Calculates the factoral of n,
        n should be an integer and n <= 0"""
    if type(n) == int and n >= 0:
        if n == 0:
            return 1
        else:
            return n * factorial(n-1)    
    else:
        raise TypeError("n has to be a positive integer or zero")

In [38]:
factorial(9)

362880

In [39]:
factorial(-5)

TypeError: n has to be a positive integer or zero

If you call this function with 4 for example i.e. factorial(4), the first thing that is checked is whether it is my positive integer.

The 'problem' now appears in the recursion step.

Now factorial(3) is called.

This call and all others also check wheteher it is a positive whole number.

But this is unnecessary: If you subtract the value '1' from a positive whole number, you get a positive whole number or 0 again. So both well-defined argument values for our function.

In [42]:
def factorial(n):
    """Calculates the factorial of n,
       n should be an integer and n <=0 """
    def inner_factorial(n):
        if n == 0:
            print('returning ...{}'.format(n))
            return 1
        else:
            ret_value = n * inner_factorial(n-1)
            print('returning ..{}'.format(ret_value))
            return ret_value
        
    if type(n) == int and n>=0:
        return inner_factorial(n)
    else:
        raise TypeError('n should be a positive int or 0')

In [43]:
factorial(3)

returning ...0
returning ..1
returning ..2
returning ..6


6

Functions as Parameters:

    Due to the fact that every parameter of a function is a reference to an object and functions are objects as well we can pass functions or better 'refrences to functions' as parameters to a function.

In [44]:
def g():
    print("Hi it's me 'g'")
    print("Thanks for calling me")
    
def f(func):
    print("Hi it's me 'f'")
    print("I will call 'func' now\n")
    func()

In [45]:
f(g)

Hi it's me 'f'
I will call 'func' now

Hi it's me 'g'
Thanks for calling me


Functions returning Functions:

    The output of a function is also a reference to an object. Therefore functions can return references to function objects.

In [47]:
def f(x):
    def g(y):
        return y + x + 3
    return g

try with arg value 10

In [48]:
f(10)

<function __main__.f.<locals>.g(y)>

In [49]:
nf1 = f(10)
nf1

<function __main__.f.<locals>.g(y)>

In [50]:
nf1(20)

33

We will implement a poynomial "factory" function.

we will start with writing a version which can create polynomials of degree2.

Image Example in folder

In [51]:
def polynomial_creator(a, b, c):

    def polynomial(x):
        return a * x**2 + b * x + c
    return polynomial

In [52]:
p1 = polynomial_creator(2,3,-1)
p2 = polynomial_creator(-1,2,1)

In [53]:
p1

<function __main__.polynomial_creator.<locals>.polynomial(x)>

In [54]:
for x in range(-2,2,1):
    print("{:5d},{:5d},{:5d}".format(x,p1(x),p2(x)))

   -2,    1,   -7
   -1,   -2,   -2
    0,   -1,    1
    1,    4,    2


We can generalize our factory function so that it can work polynomials of arbitray degree:

Image example in folder

In [55]:
def polynomial_creator(*coefficients):
    """coefficients are in the form a_n,...a_1,a_0
    """
    def polynomial(x):
        res = 0
        for index, coeff in enumerate(coefficients[::-1]):
            res += coeff * x** index
        return res
    return polynomial

p1 = polynomial_creator(4)
p2 = polynomial_creator(2,4)
p3 = polynomial_creator(1,8,-1,3,2)
p4 = polynomial_creator(-1,2,1)
        

In [56]:
for x in range(-2,2,1):
    print(x,p1(x),p2(x),p3(x),p4(x))

-2 4 0 -56 -7
-1 4 2 -9 -2
0 4 4 2 1
1 4 6 13 2


The function p3 implements, for example, the following polynomial:

        p3(x)=x^4 + 8 * x^3 - x^2 + 3 * x + 2

    The polynomial function inside of our decorator polynomial_creator can be implemented mor efficiently. 

    We can factorize it in a way so that it doesnt need any exponentiation.

    Factorized version of a general polynomial without exponentiation:

        res = (...(an * x + an_1) * x + ... + a1) * x + a0

In [57]:
def polynomial_creator(*coeffs):
    """coefficients are in the form a_n,a_n_1...a_1,a_0
    """
    def polynomial(x):
        res = coeffs[0]
        for i in range(1, len(coeffs)):
            res = res * x +coeffs[i]
        return res
    return polynomial

p1 = polynomial_creator(4)
p2 = polynomial_creator(2,4)
p3 = polynomial_creator(1,8,-1,3,2)
p4 = polynomial_creator(-1,2,1)
        

In [58]:
for x in range(-2,2,1):
    print(x,p1(x),p2(x),p3(x),p4(x))

-2 4 0 -56 -7
-1 4 2 -9 -2
0 4 4 2 1
1 4 6 13 2


A Simple Decorator:

In [64]:
def our_decorator(func):
    def function_wrapper(x):
        print("Before calling " + func.__name__)
        func(x)
        print("After calling " + func.__name__)
        
    return function_wrapper

In [65]:
def foo(x):
    print("Hi, foo has been called with " + str(x))

In [66]:
print("We call foo before decoration")

foo("Hi")

We call foo before decoration
Hi, foo has been called with Hi


In [67]:
print("We now decorate foo with f:")
foo = our_decorator(foo)

We now decorate foo with f:


In [68]:
print("We Call foo after decoration:")
foo(42)

We Call foo after decoration:
Before calling foo
Hi, foo has been called with 42
After calling foo


If you look at the output of the previous program, you can see what's going on. After the decoration "foo = our_decorator(foo)", foo is a reference to the 'function_wrapper'. 'foo' will be called inside of 'function_wrapper', but before and after the call some additional code will be executed, i.e. in our case two print functions.

The Usual Syntax for Decorators in Python:

The decotartion occurs in the line before the function header. The"@" is followed by the decorator function name.

We eill rewrite now our  initial example. Instead of writing the statement 

        foo = our_decorator(foo)
    
    we can write

        @our_decorator

In [69]:
def our_decorator(func):
    def function_wrapper(x):
        print("Before calling "+ func.__name__)
        func(x)
        print("After calling "+ func.__name__)
    return function_wrapper
@our_decorator
def foo(x)    :
    print("hi, foo has been called with " + str(x))
    
foo("Hi")

Before calling foo
hi, foo has been called with Hi
After calling foo


It is also possible to decorate third party functions, e.g. functions we import from a module.

We can't use the Python syntax with the "at" sign in this case:

Use cases for decorators:

Example 1 (Checking Arguments with a Decorator)

In [72]:
def argument_test_natural_number(f):
    def helper(x):
        if type(x) == int and x > 0:
            return f(x)
        else:
            raise Exception("Argument is not an integer")
    return helper

In [73]:
@argument_test_natural_number
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

In [74]:
for i in range(1,10):
    print(i,factorial(i))

1 1
2 2
3 6
4 24
5 120
6 720
7 5040
8 40320
9 362880


In [75]:
factorial(-1)

Exception: Argument is not an integer

Example 2:

Counting Function Calls with Decorators The Following
example uses a decorator to count the number of times a function has been called. To be precise, we can use this decoratore solely for functions with exactly one parameter: